# Loops #

In this script, time-series data is extracted from csv files, then combined to a plot.

Configure settings in [setup/config.yaml](../setup/config.yaml). Cf. [template-config.yaml](../templates/template-config.yaml) for an example.

In [ ]:
'''FORCE AUTORELOAD IMPORTS'''
# see https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
'''IMPORTS'''
import os
import sys

# NOTE: use this if the above reload commands do not work.
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key]

os.chdir(os.path.dirname(_dh[0]))
sys.path.insert(0, os.getcwd())

from src.thirdparty.render import *

from src.setup import config
from src.steps import *

In [ ]:
'''SETUP'''
PATH_USER_CONFIG = 'setup/config.yaml'
config.set_user_config(PATH_USER_CONFIG);

In [ ]:
'''READ AND COMBINE DATA'''
data_pressure, data_volume = step_read_data()
data = step_combine_data(data_pressure, data_volume)
del data_pressure, data_volume;

In [ ]:
'''PROCESS DATA'''
# NOTE: determine cycles based on pressure-peaks
data = step_compute_extremes(data, quantities=['pressure', 'volume'])
data = step_recognise_cycles(data, quantity='pressure')
if config.PROCESS_CONFIG.cycles.remove_bad:
    data = step_removed_marked_sections(data)
data = step_fit_curve(data, quantity='pressure');

In [ ]:
'''OUTPUT TABLES'''
step_output_tables(data);

In [ ]:
'''OUTPUT TIME PLOTS'''
plt_p, plt_v = step_output_time_plots(data)
display(plt_p)
display(plt_v);

In [ ]:
'''OUTPUT LOOP PLOTS'''
plt = step_output_loop_plot(data)
display(plt);